In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pickle as pickle
import glob
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error


from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# /content/drive/MyDrive/Heineken Data Scientist/LSTM Sales Forecasting FB/train_data_Rennes.csv
# /content/drive/MyDrive/Heineken Data Scientist/LSTM Sales Forecasting FB/test_data_Rennes.csv
# /content/drive/MyDrive/Heineken Data Scientist/LSTM Sales Forecasting FB/Rennes_data from 201700 to 202216.csv

In [3]:
path = "/content/drive/MyDrive/Colab Notebooks/james project/LSTM Sales Forecasting FB/Rennes_data from 201700 to 202210.csv"

df = pd.read_csv(path, sep = ';')
df

,divPrep,article,libelle,"2017, 01","2017, 02","2017, 03","2017, 04","2017, 05","2017, 06","2017, 07",...,"2022, 04","2022, 05","2022, 06","2022, 07","2022, 08","2022, 09","2022, 10",nb first zeros,duree_historique,categorie
0,SJ04,10074,EVIAN VC 12X100CL,252,744,108,144,216,276,156,...,108,36,156,96,48,60,72,0,271,A
1,SJ06,10074,EVIAN VC 12X100CL,108,336,276,216,312,420,180,...,132,108,72,132,144,240,204,0,271,A
2,SJ05,10074,EVIAN VC 12X100CL,96,36,96,60,36,156,72,...,24,24,0,0,12,48,24,0,271,A
3,SJ0P,10074,EVIAN VC 12X100CL,1044,1008,1320,1824,1200,1140,864,...,672,360,180,492,684,636,852,0,271,A
4,SJ0P,10076,VITTEL GRANDE SOURCE VC 12X100CL,1080,912,816,876,1248,2100,1836,...,564,552,816,996,1356,972,876,0,271,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,SJ0P,116902,SAUTERNES AOP CAZALIS CLOTTE BIO 2016 75,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,239,32,B
1263,SJ0P,116938,EDELWEISS 5°3 FUT BLADE 8L,0,0,0,0,0,0,0,...,160,200,160,248,176,272,256,73,198,A
1264,SJ0P,117064,APEROL BARBIERI 12°5 70CL,11,13,6,9,13,21,20,...,66,135,145,146,166,136,111,0,271,B
1265,SJ0P,117112,PLANCOET PLATE PET 6X100CL,0,0,0,0,0,0,0,...,360,360,330,390,360,396,390,245,26,B


In [4]:
df[df['article'] == 11622][['divPrep',	'article', '2017, 11', '2018, 11','2019, 11','2020, 11','2021, 11',]]

,divPrep,article,"2017, 11","2018, 11","2019, 11","2020, 11","2021, 11"
134,SJ05,11622,330,240,210,360,0
135,SJ0P,11622,240,570,1020,780,0
136,SJ04,11622,570,660,690,420,0


In [5]:
df_c = df.copy()

df_c.loc[(df_c['divPrep'] == 'SJ04') &
               (df_c['article'] == 11622), '2021, 11'] = 500

df_c.loc[(df_c['divPrep'] == 'SJ05') &
               (df_c['article'] == 11622), '2021, 11'] = 500

df_c.loc[(df_c['divPrep'] == 'SJ0P') &
               (df_c['article'] == 11622), '2021, 11'] = 1500

df_c[df_c['article'] == 11622][['divPrep',	'article', '2017, 11', '2018, 11','2019, 11','2020, 11','2021, 11',]]

,divPrep,article,"2017, 11","2018, 11","2019, 11","2020, 11","2021, 11"
134,SJ05,11622,330,240,210,360,500
135,SJ0P,11622,240,570,1020,780,1500
136,SJ04,11622,570,660,690,420,500


In [6]:
df_c

,divPrep,article,libelle,"2017, 01","2017, 02","2017, 03","2017, 04","2017, 05","2017, 06","2017, 07",...,"2022, 04","2022, 05","2022, 06","2022, 07","2022, 08","2022, 09","2022, 10",nb first zeros,duree_historique,categorie
0,SJ04,10074,EVIAN VC 12X100CL,252,744,108,144,216,276,156,...,108,36,156,96,48,60,72,0,271,A
1,SJ06,10074,EVIAN VC 12X100CL,108,336,276,216,312,420,180,...,132,108,72,132,144,240,204,0,271,A
2,SJ05,10074,EVIAN VC 12X100CL,96,36,96,60,36,156,72,...,24,24,0,0,12,48,24,0,271,A
3,SJ0P,10074,EVIAN VC 12X100CL,1044,1008,1320,1824,1200,1140,864,...,672,360,180,492,684,636,852,0,271,A
4,SJ0P,10076,VITTEL GRANDE SOURCE VC 12X100CL,1080,912,816,876,1248,2100,1836,...,564,552,816,996,1356,972,876,0,271,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,SJ0P,116902,SAUTERNES AOP CAZALIS CLOTTE BIO 2016 75,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,239,32,B
1263,SJ0P,116938,EDELWEISS 5°3 FUT BLADE 8L,0,0,0,0,0,0,0,...,160,200,160,248,176,272,256,73,198,A
1264,SJ0P,117064,APEROL BARBIERI 12°5 70CL,11,13,6,9,13,21,20,...,66,135,145,146,166,136,111,0,271,B
1265,SJ0P,117112,PLANCOET PLATE PET 6X100CL,0,0,0,0,0,0,0,...,360,360,330,390,360,396,390,245,26,B


In [54]:
def encode_DivPrep_Article_function(df_sales_data):
#     print('################### encode_DivPrep_Article_function ###################') 

    sales_lib = df_sales_data.drop(['duree_historique'], axis = 1)
    
    # we concatenate divprep and article
    sales_lib['divPrep_article'] = sales_lib.apply(lambda row: row['divPrep']+'_'+str(int(row['article'])) #create a column at the end
                                               , axis = 1)
    #print("ddddd",['divPrep', 'article', 'libelle' , 'divPrep_article', 'categorie', 'nb first zeros'] +  #['divPrep', 'article', 'libelle', 'divPrep_article', 'categorie', 'nb first zeros', '2017, 01',....]
    #                                          list(sales_lib.columns[3:-3]))
    sales_lib = sales_lib.reindex(columns=['divPrep', 'article', 'libelle' , 'divPrep_article', 'categorie', 'nb first zeros'] + 
                                              list(sales_lib.columns[3:-3]))
    encoder = OrdinalEncoder()
    # transform data
    # then we encode the divprep/article
    #print("fit transform",np.asarray(sales_lib['divPrep_article']).reshape(-1, 1)).tolist() #['SJ04_10074' 'SJ06_10074' 'SJ05_10074' ... 'SJ0P_117064' 'SJ0P_117112' 'SJ0P_117116']
    #print("fit transform",encoder.fit_transform(np.asarray(sales_lib['divPrep_article']).reshape(-1, 1)).tolist()) #[[3.0], [295.0], [150.0],...]
    sales_lib['divPrep_article'] = [x[0] for x in encoder.fit_transform(np.asarray(sales_lib['divPrep_article']).reshape(-1, 1)).tolist()]
    sales_lib.set_index(['divPrep_article'], inplace=True) #If inplace True, modifies the DataFrame in place (do not create a new object).
    sales_ = sales_lib.drop(['article', 'divPrep', 'libelle', 'categorie', 'nb first zeros'], axis = 1)
    print(len(sales_.index.unique())) #1267
#     display('sales_lib : ', sales_lib)
#     display('sales_ : ', sales_)
    return sales_lib, sales_

# Melt function
def melt_function(sales_):
#     print('################### melt_function ###################') 
    print("aaa",sales_)
    melt_data = sales_.reset_index().melt(id_vars='divPrep_article', var_name='Week', value_name='Sales').copy()
    print("melt",melt_data)
    melt_data['Year'] = melt_data['Week'].apply(lambda row: row[:4]).astype(int)
    melt_data['Week'] = melt_data['Week'].apply(lambda row: row[6:]).astype(int)
    melt_data['Month'] = pd.to_datetime(melt_data.Year * 1000 + melt_data.Week * 10 + 0, format='%Y%W%w').dt.to_period('W').dt.start_time.dt.month
    
#     display('melt_data : ', melt_data)
    return melt_data


# drop first zeros function
def drop_first_zeros_function(sales_lib,melt_data):
    print('################### drop_first_zeros_function ###################')
    nfz = sales_lib[sales_lib['nb first zeros'] != 0]['nb first zeros']
    total_rows_dropped = 0 
    nb_row_before = melt_data.shape[0]
    
    melt_data_no_first_0 = melt_data.copy()
    
    for i in range(0,len(nfz)):
#         print('drop first ' +str(nfz.values[i])+ ' null sales from org_com_article : ' + str(nfz.index[i]))
        melt_data_no_first_0.drop(index=list(melt_data_no_first_0[(melt_data_no_first_0['divPrep_article'] == nfz.index[i])].iloc[:nfz.values[i],:].index),
                   inplace = True)
        total_rows_dropped += nfz.values[i]

    print('nb of articles with nfz != 0 :' +str(len(nfz))) 
    print('total_rows_dropped :' +str(total_rows_dropped))
    print('nb_row_before :', nb_row_before)
    print(f'check : {nb_row_before} - {total_rows_dropped} = {nb_row_before - total_rows_dropped}')
    print('melt_data_no_first_0.shape : ', melt_data_no_first_0.shape)
    return melt_data_no_first_0


# Feature Engineering function 
def fe_function(melt_data, last_art):
#     print('################### fe_function ###################')
    melt_data_FE6 = melt_data.copy()
    
    # Create Sales Lag 1 to 6
    for i in range(1,6+1):
        melt_data_FE6['Sales_lag_'+str(i)] = melt_data_FE6.groupby(['divPrep_article'])['Sales'].shift(i)

    # Create expanding mean from sales lag 1
    melt_data_FE6['Sales_lag_1_expanding_8'] = add_expanding_mean1(melt_data_FE6)

    # Create rolling mean from sales lag 1 
    melt_data_FE6['Sales_lag_1_rolling_4'] = add_rolling_mean1(melt_data_FE6)

    print('FE 6')
#     display(melt_data_FE6)
#     display(melt_data_FE6[melt_data_FE6['divPrep_article'] == last_art].head(10))
    # the fact to create rolling and expanding mean, it will create some NaN values.
    melt_data_FE6 = melt_data_FE6.dropna()
    melt_data_FE6.reset_index(drop = True, inplace = True)
#     display(melt_data_FE6[melt_data_FE6['divPrep_article'] == last_art].head(10))
    return melt_data_FE6


In [56]:
# Encode Org_com / Article 
sales_lib, sales_ = encode_DivPrep_Article_function(df_c)
display(sales_lib)

1267


,divPrep,article,libelle,categorie,nb first zeros,"2017, 01","2017, 02","2017, 03","2017, 04","2017, 05",...,"2022, 01","2022, 02","2022, 03","2022, 04","2022, 05","2022, 06","2022, 07","2022, 08","2022, 09","2022, 10"
divPrep_article,,,,,,,,,,,,,,,,,,,,,
3.0,SJ04,10074,EVIAN VC 12X100CL,A,0,252,744,108,144,216,...,132,72,12,108,36,156,96,48,60,72
295.0,SJ06,10074,EVIAN VC 12X100CL,A,0,108,336,276,216,312,...,120,36,84,132,108,72,132,144,240,204
150.0,SJ05,10074,EVIAN VC 12X100CL,A,0,96,36,96,60,36,...,48,24,48,24,24,0,0,12,48,24
418.0,SJ0P,10074,EVIAN VC 12X100CL,A,0,1044,1008,1320,1824,1200,...,540,468,444,672,360,180,492,684,636,852
419.0,SJ0P,10076,VITTEL GRANDE SOURCE VC 12X100CL,A,0,1080,912,816,876,1248,...,600,324,588,564,552,816,996,1356,972,876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835.0,SJ0P,116902,SAUTERNES AOP CAZALIS CLOTTE BIO 2016 75,B,239,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
836.0,SJ0P,116938,EDELWEISS 5°3 FUT BLADE 8L,A,73,0,0,0,0,0,...,184,160,264,160,200,160,248,176,272,256
837.0,SJ0P,117064,APEROL BARBIERI 12°5 70CL,B,0,11,13,6,9,13,...,103,77,87,66,135,145,146,166,136,111


In [59]:
# Lets Melt !  
melt_data = melt_function(sales_)
#display(melt_data)

aaa                  2017, 01  2017, 02  2017, 03  2017, 04  2017, 05  2017, 06  \
divPrep_article                                                               
3.0                   252       744       108       144       216       276   
295.0                 108       336       276       216       312       420   
150.0                  96        36        96        60        36       156   
418.0                1044      1008      1320      1824      1200      1140   
419.0                1080       912       816       876      1248      2100   
...                   ...       ...       ...       ...       ...       ...   
835.0                   0         0         0         0         0         0   
836.0                   0         0         0         0         0         0   
837.0                  11        13         6         9        13        21   
838.0                   0         0         0         0         0         0   
839.0                   0         0         0   

In [ ]:
# drop first zeros 
melt_data_no_first_0 = drop_first_zeros_function(sales_lib, melt_data) 

In [10]:
melt_data_no_first_0 = melt_data_no_first_0.reset_index(drop=True)
melt_data_no_first_0

,divPrep_article,Week,Sales,Year,Month
0,3.0,1,252,2017,1
1,295.0,1,108,2017,1
2,150.0,1,96,2017,1
3,418.0,1,1044,2017,1
4,419.0,1,1080,2017,1
...,...,...,...,...,...
320250,835.0,10,0,2022,3
320251,836.0,10,256,2022,3
320252,837.0,10,111,2022,3
320253,838.0,10,390,2022,3


# Training dataset

In [11]:
# df_train = melt_data_no_first_0.iloc[:row_data_202211,:]
# df_train

In [12]:
# X_train, y_train = df_train.iloc[:-1385:].drop('Sales', axis = 1), df_train.iloc[:-1385:]['Sales']
# display(X_train, y_train)

In [13]:
# X_eval, y_eval = df_train.iloc[-1385:,:].drop('Sales', axis = 1), df_train.iloc[-1385:,:]['Sales']
# display(X_eval, y_eval)

In [14]:
# df_test = melt_data_no_first_0.iloc[row_data_202211:,:].reset_index(drop=True)
# df_test

In [15]:
# X_test, y_test = df_test.iloc[:1385:].drop('Sales', axis = 1), df_test.iloc[:1385:]['Sales']
# display(X_test, y_test)

In [16]:
# sequence_length = 1 # size of features seq
# batch_size # size of labels seq ? 

# dataset_train = keras.preprocessing.timeseries_dataset_from_array(
#     X_train,
#     y_train,
#     sequence_length=sequence_length,
#     batch_size=batch_size,
# )

# dataset_train

In [17]:
# for inputs, targets in dataset_train:
#   print("Input:", inputs.numpy(), "Target:", targets.numpy())

In [18]:
sales_lib

,divPrep,article,libelle,categorie,nb first zeros,"2017, 01","2017, 02","2017, 03","2017, 04","2017, 05",...,"2022, 01","2022, 02","2022, 03","2022, 04","2022, 05","2022, 06","2022, 07","2022, 08","2022, 09","2022, 10"
divPrep_article,,,,,,,,,,,,,,,,,,,,,
3.0,SJ04,10074,EVIAN VC 12X100CL,A,0,252,744,108,144,216,...,132,72,12,108,36,156,96,48,60,72
295.0,SJ06,10074,EVIAN VC 12X100CL,A,0,108,336,276,216,312,...,120,36,84,132,108,72,132,144,240,204
150.0,SJ05,10074,EVIAN VC 12X100CL,A,0,96,36,96,60,36,...,48,24,48,24,24,0,0,12,48,24
418.0,SJ0P,10074,EVIAN VC 12X100CL,A,0,1044,1008,1320,1824,1200,...,540,468,444,672,360,180,492,684,636,852
419.0,SJ0P,10076,VITTEL GRANDE SOURCE VC 12X100CL,A,0,1080,912,816,876,1248,...,600,324,588,564,552,816,996,1356,972,876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835.0,SJ0P,116902,SAUTERNES AOP CAZALIS CLOTTE BIO 2016 75,B,239,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
836.0,SJ0P,116938,EDELWEISS 5°3 FUT BLADE 8L,A,73,0,0,0,0,0,...,184,160,264,160,200,160,248,176,272,256
837.0,SJ0P,117064,APEROL BARBIERI 12°5 70CL,B,0,11,13,6,9,13,...,103,77,87,66,135,145,146,166,136,111


In [19]:
a = sales_.reset_index()
a = a.sort_values(['divPrep_article'])
a['divPrep_article'] = a['divPrep_article'].astype(str)
a = a.set_index('divPrep_article')
a

,"2017, 01","2017, 02","2017, 03","2017, 04","2017, 05","2017, 06","2017, 07","2017, 08","2017, 09","2017, 10",...,"2022, 01","2022, 02","2022, 03","2022, 04","2022, 05","2022, 06","2022, 07","2022, 08","2022, 09","2022, 10"
divPrep_article,,,,,,,,,,,,,,,,,,,,,
0.0,740,1040,1280,780,1180,1040,700,900,1040,1100,...,760,800,780,900,920,940,1140,1220,1480,1080
1.0,6390,9360,7440,9690,8370,9450,7590,9690,8700,8340,...,5250,5700,5250,5610,5220,5490,5640,5640,6000,7290
2.0,990,960,1320,750,1410,990,1800,1020,1380,1140,...,540,630,270,660,510,780,510,750,360,1740
3.0,252,744,108,144,216,276,156,180,108,348,...,132,72,12,108,36,156,96,48,60,72
4.0,156,852,108,168,228,264,228,96,216,132,...,0,24,0,72,60,180,144,132,72,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262.0,180,120,144,204,108,312,276,168,120,408,...,108,108,156,48,216,120,180,216,372,240
1263.0,8,10,16,16,15,15,18,20,15,13,...,8,5,9,6,8,16,15,26,22,14
1264.0,0,0,18,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
train_df = a.T
train_df

divPrep_article,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,1257.0,1258.0,1259.0,1260.0,1261.0,1262.0,1263.0,1264.0,1265.0,1266.0
"2017, 01",740,6390,990,252,156,396,0,924,720,120,...,324,384,780,432,192,180,8,0,18,408
"2017, 02",1040,9360,960,744,852,444,0,1128,510,120,...,324,156,648,324,264,120,10,0,48,768
"2017, 03",1280,7440,1320,108,108,264,0,234,270,40,...,252,240,780,216,168,144,16,18,6,600
"2017, 04",780,9690,750,144,168,456,0,2226,300,140,...,228,144,588,384,132,204,16,0,54,456
"2017, 05",1180,8370,1410,216,228,396,0,2046,780,80,...,228,72,552,264,228,108,15,6,30,648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"2022, 06",940,5490,780,156,180,312,492,3174,300,100,...,240,240,528,216,324,120,16,0,18,552
"2022, 07",1140,5640,510,96,144,384,828,3438,270,120,...,300,228,660,264,420,180,15,0,36,480
"2022, 08",1220,5640,750,48,132,360,1140,1950,330,60,...,360,240,540,144,408,216,26,0,12,240
"2022, 09",1480,6000,360,60,72,264,948,2148,390,20,...,456,480,936,504,528,372,22,0,42,264


In [21]:
sc = MinMaxScaler(feature_range = (0, 1))

train_df_scaled = train_df.copy()

train_df_scaled_ = sc.fit_transform(train_df.copy())
display(train_df_scaled_)

# pd.DataFrame(train_df_scaled_, columns = train_df.columns.to_list())
train_df_scaled.iloc[:,:] = train_df_scaled_
train_df_scaled

array([[0.33035714, 0.43292683, 0.17010309, ..., 0.        , 0.00437956,
        0.09139785],
       [0.46428571, 0.63414634, 0.16494845, ..., 0.        , 0.01167883,
        0.17204301],
       [0.57142857, 0.50406504, 0.22680412, ..., 0.1875    , 0.00145985,
        0.1344086 ],
       ...,
       [0.54464286, 0.38211382, 0.12886598, ..., 0.        , 0.00291971,
        0.05376344],
       [0.66071429, 0.40650407, 0.06185567, ..., 0.        , 0.01021898,
        0.05913978],
       [0.48214286, 0.49390244, 0.29896907, ..., 0.        , 0.02481752,
        0.09139785]])

divPrep_article,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,1257.0,1258.0,1259.0,1260.0,1261.0,1262.0,1263.0,1264.0,1265.0,1266.0
"2017, 01",0.330357,0.432927,0.170103,0.287671,0.183099,0.370787,0.000000,0.177804,0.139535,0.171429,...,0.296703,0.432432,0.342105,0.387097,0.150943,0.272727,0.074766,0.0000,0.004380,0.091398
"2017, 02",0.464286,0.634146,0.164948,0.849315,1.000000,0.415730,0.000000,0.218377,0.098837,0.171429,...,0.296703,0.175676,0.284211,0.290323,0.207547,0.181818,0.093458,0.0000,0.011679,0.172043
"2017, 03",0.571429,0.504065,0.226804,0.123288,0.126761,0.247191,0.000000,0.040573,0.052326,0.057143,...,0.230769,0.270270,0.342105,0.193548,0.132075,0.218182,0.149533,0.1875,0.001460,0.134409
"2017, 04",0.348214,0.656504,0.128866,0.164384,0.197183,0.426966,0.000000,0.436754,0.058140,0.200000,...,0.208791,0.162162,0.257895,0.344086,0.103774,0.309091,0.149533,0.0000,0.013139,0.102151
"2017, 05",0.526786,0.567073,0.242268,0.246575,0.267606,0.370787,0.000000,0.400955,0.151163,0.114286,...,0.208791,0.081081,0.242105,0.236559,0.179245,0.163636,0.140187,0.0625,0.007299,0.145161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"2022, 06",0.419643,0.371951,0.134021,0.178082,0.211268,0.292135,0.222826,0.625298,0.058140,0.142857,...,0.219780,0.270270,0.231579,0.193548,0.254717,0.181818,0.149533,0.0000,0.004380,0.123656
"2022, 07",0.508929,0.382114,0.087629,0.109589,0.169014,0.359551,0.375000,0.677804,0.052326,0.171429,...,0.274725,0.256757,0.289474,0.236559,0.330189,0.272727,0.140187,0.0000,0.008759,0.107527
"2022, 08",0.544643,0.382114,0.128866,0.054795,0.154930,0.337079,0.516304,0.381862,0.063953,0.085714,...,0.329670,0.270270,0.236842,0.129032,0.320755,0.327273,0.242991,0.0000,0.002920,0.053763
"2022, 09",0.660714,0.406504,0.061856,0.068493,0.084507,0.247191,0.429348,0.421241,0.075581,0.028571,...,0.417582,0.540541,0.410526,0.451613,0.415094,0.563636,0.205607,0.0000,0.010219,0.059140


Generating Sequence on scaled data

In [22]:
timesteps=14

X_train_sc = []
y_train_sc = []

for i in range(timesteps, 271):
    X_train_sc.append((train_df_scaled.iloc[i-timesteps:i,:]))
    y_train_sc.append(train_df_scaled.iloc[i,:]) 
  
X_train_sc = np.array(X_train_sc, dtype = 'float16')
y_train_sc = np.array(y_train_sc, dtype = 'float16')

display(X_train_sc)
display(y_train_sc)

array([[[0.3303  , 0.4329  , 0.17    , ..., 0.      , 0.00438 ,
         0.0914  ],
        [0.4644  , 0.6343  , 0.1649  , ..., 0.      , 0.01168 ,
         0.172   ],
        [0.5713  , 0.504   , 0.2268  , ..., 0.1875  , 0.00146 ,
         0.1344  ],
        ...,
        [0.5     , 0.6787  , 0.17    , ..., 0.      , 0.014595,
         0.172   ],
        [0.5444  , 0.5693  , 0.17    , ..., 0.      , 0.02043 ,
         0.1344  ],
        [0.5806  , 0.6646  , 0.3298  , ..., 0.      , 0.007298,
         0.2097  ]],

       [[0.4644  , 0.6343  , 0.1649  , ..., 0.      , 0.01168 ,
         0.172   ],
        [0.5713  , 0.504   , 0.2268  , ..., 0.1875  , 0.00146 ,
         0.1344  ],
        [0.3481  , 0.6567  , 0.1289  , ..., 0.      , 0.01314 ,
         0.1022  ],
        ...,
        [0.5444  , 0.5693  , 0.17    , ..., 0.      , 0.02043 ,
         0.1344  ],
        [0.5806  , 0.6646  , 0.3298  , ..., 0.      , 0.007298,
         0.2097  ],
        [0.634   , 0.7075  , 0.201   , ..., 0.  

array([[0.634   , 0.7075  , 0.201   , ..., 0.      , 0.03503 , 0.2312  ],
       [0.527   , 0.6626  , 0.1855  , ..., 0.      , 0.1416  , 0.1882  ],
       [0.527   , 0.6953  , 0.2732  , ..., 0.      , 0.1445  , 0.1398  ],
       ...,
       [0.5444  , 0.382   , 0.1289  , ..., 0.      , 0.00292 , 0.05377 ],
       [0.6606  , 0.4065  , 0.06186 , ..., 0.      , 0.010216, 0.05914 ],
       [0.4822  , 0.494   , 0.299   , ..., 0.      , 0.02481 , 0.0914  ]],
      dtype=float16)

## Model

In [23]:
# modeling with scaled data

In [24]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=128, input_shape = (np.array(X_train_sc).shape[1], np.array(X_train_sc).shape[2]), return_sequences=True)))
model2.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64,input_shape = (np.array(X_train_sc).shape[1], np.array(X_train_sc).shape[2]))))
model2.add(tf.keras.layers.Dense(1267))

model2.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.001))

model2.fit(X_train_sc, y_train_sc, epochs=50)

Epoch 1/50
9/9 [==============================] - 11s 13ms/step - loss: 0.0645
Epoch 2/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0302
Epoch 3/50
9/9 [==============================] - 0s 12ms/step - loss: 0.0271
Epoch 4/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0245
Epoch 5/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0232
Epoch 6/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0219
Epoch 7/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0207
Epoch 8/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0204
Epoch 9/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0202
Epoch 10/50
9/9 [==============================] - 0s 10ms/step - loss: 0.0189
Epoch 11/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0185
Epoch 12/50
9/9 [==============================] - 0s 11ms/step - loss: 0.0196
Epoch 13/50
9/9 [==============================] - 0s 11ms/s

In [25]:
inputs= train_df_scaled[-timesteps:]
display(inputs)

X_test=[]
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
X_test

divPrep_article,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,1257.0,1258.0,1259.0,1260.0,1261.0,1262.0,1263.0,1264.0,1265.0,1266.0
"2021, 49",0.437500,0.367886,0.211340,0.232877,0.084507,0.269663,0.260870,0.078759,0.058140,0.000000,...,0.142857,0.243243,0.221053,0.107527,0.216981,0.272727,0.074766,0.0000,0.019708,0.026882
"2021, 50",0.303571,0.380081,0.154639,0.013699,0.394366,0.404494,0.576087,0.435561,0.058140,0.114286,...,0.285714,0.283784,0.357895,0.182796,0.264151,0.163636,0.065421,0.0625,0.055474,0.048387
"2021, 51",0.285714,0.325203,0.056701,0.123288,0.098592,0.224719,0.239130,0.527446,0.063953,0.000000,...,0.098901,0.189189,0.194737,0.182796,0.207547,0.181818,0.121495,0.0625,0.024818,0.075269
"2021, 52",0.250000,0.264228,0.077320,0.041096,0.056338,0.202247,0.103261,0.507160,0.034884,0.000000,...,0.186813,0.121622,0.131579,0.064516,0.141509,0.072727,0.037383,0.0000,0.002920,0.016129
"2022, 01",0.339286,0.355691,0.092784,0.150685,0.000000,0.191011,0.206522,0.177804,0.029070,0.028571,...,0.186813,0.202703,0.173684,0.150538,0.226415,0.163636,0.074766,0.0000,0.017518,0.026882
"2022, 02",0.357143,0.386179,0.108247,0.082192,0.028169,0.337079,0.554348,0.317422,0.046512,0.000000,...,0.219780,0.256757,0.157895,0.279570,0.141509,0.163636,0.046729,0.0000,0.018978,0.048387
"2022, 03",0.348214,0.355691,0.046392,0.013699,0.000000,0.191011,0.288043,0.948687,0.250000,0.028571,...,0.142857,0.243243,0.157895,0.333333,0.188679,0.236364,0.084112,0.0000,0.002920,0.037634
"2022, 04",0.401786,0.380081,0.113402,0.123288,0.084507,0.202247,0.364130,0.078759,0.034884,0.057143,...,0.175824,0.148649,0.152632,0.118280,0.084906,0.072727,0.056075,0.0000,0.011679,0.037634
"2022, 05",0.410714,0.353659,0.087629,0.041096,0.070423,0.280899,0.385870,0.534606,0.034884,0.057143,...,0.197802,0.270270,0.178947,0.247312,0.226415,0.327273,0.074766,0.0000,0.008759,0.032258
"2022, 06",0.419643,0.371951,0.134021,0.178082,0.211268,0.292135,0.222826,0.625298,0.058140,0.142857,...,0.219780,0.270270,0.231579,0.193548,0.254717,0.181818,0.149533,0.0000,0.004380,0.123656


array([[[0.4375    , 0.36788618, 0.21134021, ..., 0.        ,
         0.01970803, 0.02688172],
        [0.30357143, 0.3800813 , 0.15463918, ..., 0.0625    ,
         0.05547445, 0.0483871 ],
        [0.28571429, 0.32520325, 0.05670103, ..., 0.0625    ,
         0.02481752, 0.07526882],
        ...,
        [0.54464286, 0.38211382, 0.12886598, ..., 0.        ,
         0.00291971, 0.05376344],
        [0.66071429, 0.40650407, 0.06185567, ..., 0.        ,
         0.01021898, 0.05913978],
        [0.48214286, 0.49390244, 0.29896907, ..., 0.        ,
         0.02481752, 0.09139785]]])

In [26]:
predictions_sc = model2.predict(X_test)
predictions_sc

array([[ 0.4815635 ,  0.3816902 ,  0.14885746, ..., -0.00203412,
         0.03111847,  0.06442918]], dtype=float32)

In [27]:
predictions_unsc = sc.inverse_transform(predictions_sc)
predictions_unsc

array([[ 1.0787023e+03,  5.6337476e+03,  8.6635040e+02, ...,
        -1.9527519e-01,  1.2789690e+02,  2.8761185e+02]], dtype=float32)

In [28]:
predictions_unsc[0]

array([ 1.0787023e+03,  5.6337476e+03,  8.6635040e+02, ...,
       -1.9527519e-01,  1.2789690e+02,  2.8761185e+02], dtype=float32)

In [29]:
path_11 = "/content/drive/MyDrive/Colab Notebooks/james project/LSTM Sales Forecasting FB/Rennes_data from 201700 to 202212.csv"
df_11 = pd.read_csv(path_11, sep = ';')
df_11 = df_11[['divPrep',	'article',	'libelle', '2022, 11']]
df_11

,divPrep,article,libelle,"2022, 11"
0,SJ04,10074,EVIAN VC 12X100CL,156
1,SJ06,10074,EVIAN VC 12X100CL,96
2,SJ05,10074,EVIAN VC 12X100CL,84
3,SJ0P,10074,EVIAN VC 12X100CL,1176
4,SJ0P,10076,VITTEL GRANDE SOURCE VC 12X100CL,1008
...,...,...,...,...
1250,SJ0P,116902,SAUTERNES AOP CAZALIS CLOTTE BIO 2016 75,0
1251,SJ0P,116938,EDELWEISS 5°3 FUT BLADE 8L,352
1252,SJ0P,117064,APEROL BARBIERI 12°5 70CL,202
1253,SJ0P,117112,PLANCOET PLATE PET 6X100CL,438


In [30]:
pred_path = "/content/drive/MyDrive/Colab Notebooks/james project/LSTM Sales Forecasting FB/replenisher_Rennes_S11_2022_on_6_weeks.xlsx"

df_preds = pd.read_excel(pred_path)
df_preds = df_preds.iloc[:,:4]
df_preds = df_preds[['divPrep',	'article', 'S11']]


df_preds

,divPrep,article,S11
0,SJ04,10074,69
1,SJ06,10074,188
2,SJ05,10074,27
3,SJ0P,10074,778
4,SJ0P,10076,1093
...,...,...,...
1262,SJ0P,116902,11
1263,SJ0P,116938,246
1264,SJ0P,117064,123
1265,SJ0P,117112,343


In [31]:
real_lgbm_df = pd.merge(df_11, df_preds, left_on = ['divPrep',	'article'], right_on = ['divPrep',	'article'], how = 'left')
real_lgbm_df

,divPrep,article,libelle,"2022, 11",S11
0,SJ04,10074,EVIAN VC 12X100CL,156,69
1,SJ06,10074,EVIAN VC 12X100CL,96,188
2,SJ05,10074,EVIAN VC 12X100CL,84,27
3,SJ0P,10074,EVIAN VC 12X100CL,1176,778
4,SJ0P,10076,VITTEL GRANDE SOURCE VC 12X100CL,1008,1093
...,...,...,...,...,...
1250,SJ0P,116902,SAUTERNES AOP CAZALIS CLOTTE BIO 2016 75,0,11
1251,SJ0P,116938,EDELWEISS 5°3 FUT BLADE 8L,352,246
1252,SJ0P,117064,APEROL BARBIERI 12°5 70CL,202,123
1253,SJ0P,117112,PLANCOET PLATE PET 6X100CL,438,343


In [32]:
sales_lib.reset_index().iloc[:,:3]

,divPrep_article,divPrep,article
0,3.0,SJ04,10074
1,295.0,SJ06,10074
2,150.0,SJ05,10074
3,418.0,SJ0P,10074
4,419.0,SJ0P,10076
...,...,...,...
1262,835.0,SJ0P,116902
1263,836.0,SJ0P,116938
1264,837.0,SJ0P,117064
1265,838.0,SJ0P,117112


In [33]:
prev_lstm = train_df_scaled.iloc[-1,:].to_frame().reset_index()
prev_lstm['lstm_pred'] = predictions_unsc[0]
prev_lstm = prev_lstm[['divPrep_article', 'lstm_pred']]
prev_lstm['divPrep_article'] = prev_lstm['divPrep_article'].astype(float)
prev_lstm

,divPrep_article,lstm_pred
0,0.0,1078.702271
1,1.0,5633.747559
2,2.0,866.350403
3,3.0,94.189514
4,4.0,132.914230
...,...,...
1262,1262.0,243.820694
1263,1263.0,15.536986
1264,1264.0,-0.195275
1265,1265.0,127.896896


In [34]:
prev_lstm_df = pd.merge(sales_lib.reset_index().iloc[:,:3], prev_lstm, left_on = ['divPrep_article'], right_on = ['divPrep_article'], how = 'left')
prev_lstm_df

,divPrep_article,divPrep,article,lstm_pred
0,3.0,SJ04,10074,94.189514
1,295.0,SJ06,10074,336.144592
2,150.0,SJ05,10074,16.442680
3,418.0,SJ0P,10074,757.164429
4,419.0,SJ0P,10076,960.023193
...,...,...,...,...
1262,835.0,SJ0P,116902,3.464149
1263,836.0,SJ0P,116938,363.425293
1264,837.0,SJ0P,117064,214.534256
1265,838.0,SJ0P,117112,239.748795


In [35]:
final_prev_df = pd.merge(real_lgbm_df, prev_lstm_df.iloc[:,1:], left_on = ['divPrep','article'], right_on = ['divPrep','article'], how = 'left')
final_prev_df['abs_err_lgbm'] = abs(final_prev_df['2022, 11'] - final_prev_df['S11'])
final_prev_df['abs_err_lstm'] = abs(final_prev_df['2022, 11'] - final_prev_df['lstm_pred'])
final_prev_df

,divPrep,article,libelle,"2022, 11",S11,lstm_pred,abs_err_lgbm,abs_err_lstm
0,SJ04,10074,EVIAN VC 12X100CL,156,69,94.189514,87,61.810486
1,SJ06,10074,EVIAN VC 12X100CL,96,188,336.144592,92,240.144592
2,SJ05,10074,EVIAN VC 12X100CL,84,27,16.442680,57,67.557320
3,SJ0P,10074,EVIAN VC 12X100CL,1176,778,757.164429,398,418.835571
4,SJ0P,10076,VITTEL GRANDE SOURCE VC 12X100CL,1008,1093,960.023193,85,47.976807
...,...,...,...,...,...,...,...,...
1250,SJ0P,116902,SAUTERNES AOP CAZALIS CLOTTE BIO 2016 75,0,11,3.464149,11,3.464149
1251,SJ0P,116938,EDELWEISS 5°3 FUT BLADE 8L,352,246,363.425293,106,11.425293
1252,SJ0P,117064,APEROL BARBIERI 12°5 70CL,202,123,214.534256,79,12.534256
1253,SJ0P,117112,PLANCOET PLATE PET 6X100CL,438,343,239.748795,95,198.251205


In [36]:
print("mae LSTM : ", mean_absolute_error(final_prev_df['2022, 11'], final_prev_df['lstm_pred']))
print("mse LSTM : ", mean_squared_error(final_prev_df['2022, 11'], final_prev_df['lstm_pred']))

print("mae LGBM : ", mean_absolute_error(final_prev_df['2022, 11'], final_prev_df['S11']))
print("mse LGBM : ", mean_squared_error(final_prev_df['2022, 11'], final_prev_df['S11']))

print("FA LGBM : ", 1 - (final_prev_df['abs_err_lgbm'].sum())/final_prev_df['2022, 11'].sum())
print("FA LSTM : ", 1 - (final_prev_df['abs_err_lstm'].sum())/final_prev_df['2022, 11'].sum())


mae LSTM :  157.38434828849458
mse LSTM :  248015.01289951935
mae LGBM :  149.18725099601593
mse LGBM :  244850.83824701194
FA LGBM :  0.748462400348227
FA LSTM :  0.7346416605511167


In [37]:
# 1 lstm 64 

# mae LSTM :  152.20643606507684
# mse LSTM :  223627.6511970084
# mae LGBM :  149.18725099601593
# mse LGBM :  244850.83824701194
# FA LGBM :  0.748462400348227
# FA LSTM :  0.7433718945569759

In [38]:
# bidirectional 1 lstm 128
# bidirectional 1 lstm 64


# mae LSTM :  148.8673716618149
# mse LSTM :  224104.60609660603
# mae LGBM :  149.18725099601593
# mse LGBM :  244850.83824701194
# FA LGBM :  0.748462400348227
# FA LSTM :  0.7490017338325963

In [39]:
final_prev_df[final_prev_df['article'] == 11622]

,divPrep,article,libelle,"2022, 11",S11,lstm_pred,abs_err_lgbm,abs_err_lstm
134,SJ05,11622,GUINNESS 4°2 FUT 30L,540,328,226.823776,212,313.176224
135,SJ04,11622,GUINNESS 4°2 FUT 30L,510,164,174.844864,346,335.155136
136,SJ0P,11622,GUINNESS 4°2 FUT 30L,1470,591,545.185852,879,924.814148


In [40]:
# 	divPrep	article	libelle	2022, 11	S11	lstm_pred	abs_err_lgbm	abs_err_lstm
# 134	SJ05	11622	GUINNESS 4°2 FUT 30L	540	328	227.694885	212	312.305115
# 135	SJ04	11622	GUINNESS 4°2 FUT 30L	510	164	179.874954	346	330.125046
# 136	SJ0P	11622	GUINNESS 4°2 FUT 30L	1470	591	672.167358	879	797.832642